In [306]:
%matplotlib inline

In [307]:
import numpy as np
import math
from numpy import expand_dims
from scipy.integrate import odeint,quad,dblquad,simps,quad_vec,nquad

In [328]:
import tensorflow as tf

import keras
from keras import backend as K
from keras import datasets, layers, models
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import optimizers
from keras.callbacks import ModelCheckpoint

from matplotlib import pyplot
import pandas as pd
from glob import glob 
import csv

from sklearn.model_selection import train_test_split
from skimage.io import imread

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from colorspacious import cspace_converter
from collections import OrderedDict

import pylab as py
import threading
import scipy

from PIL import Image

from scipy import constants
from scipy import interpolate
from scipy import signal
from scipy import ndimage
from scipy.linalg import block_diag
import sympy

In [309]:
# concatenate the CSV Files of the 
train_labels = sorted(glob('/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label*.csv'))
print(train_labels)
# write header of csv file
header = ['id','appliance']
with open('data_cnn/header.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)

# create dataset
fout = open('data_cnn/train_labels.csv','a')

for line in open('data_cnn/header.csv'):
    fout.write(line)

for i in range(5):
    file = open(train_labels[i])
#     file.__next__()
    for line in file:
        fout.write(line)
    file.close()
fout.close()
# ######### weg got our dataset train_labels ##########

['/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label_10cm_5cm_focus_Siemensstar_10cm_1_METAL_L_Z96.50_.csv', '/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label_10cm_5cm_focus_Siemensstar_10cm_1_METAL_V_Z96.50_.csv', '/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label_10cm_5cm_focus_Siemensstar_10cm_1_phase_L_Z96.50_.csv', '/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label_10cm_5cm_focus_Siemensstar_10cm_1p5_metal_V_Z96.50_.csv', '/home/alex/ml_master/virt/master_ml/cnn_fourier/data_cnn/label_10cm_5cm_focus_Siemensstar_10cm_2_phase_V_Z96.50_.csv']


In [310]:
# Read the Dataset we created ####
train_df = pd.read_csv('data_cnn/train_labels.csv', index_col=0)
train_df['amplitude'] = train_df.index.map(lambda id: f'data_cnn/train/{id}_ampl.png')
train_df['phase'] = train_df.index.map(lambda id: f'data_cnn/train/{id}_phase.png')
print(train_df.head())

                                                    appliance  \
id                                                              
10cm_5cm_focus_Siemensstar_10cm_1_METAL_L_Z96.50            1   
10cm_5cm_focus_Siemensstar_10cm_1_METAL_V_Z96.50            1   
10cm_5cm_focus_Siemensstar_10cm_1_phase_L_Z96.50            1   
10cm_5cm_focus_Siemensstar_10cm_1p5_metal_V_Z96.50          1   
10cm_5cm_focus_Siemensstar_10cm_2_phase_V_Z96.50            1   

                                                                                            amplitude  \
id                                                                                                      
10cm_5cm_focus_Siemensstar_10cm_1_METAL_L_Z96.50    data_cnn/train/10cm_5cm_focus_Siemensstar_10cm...   
10cm_5cm_focus_Siemensstar_10cm_1_METAL_V_Z96.50    data_cnn/train/10cm_5cm_focus_Siemensstar_10cm...   
10cm_5cm_focus_Siemensstar_10cm_1_phase_L_Z96.50    data_cnn/train/10cm_5cm_focus_Siemensstar_10cm...   
10cm_5cm_focus_Siem

In [311]:
def read_data(file_paths, img_rows, img_cols,  channels):
    """
      Reads the spectogram files from disk and normalizes the pixel values
        @params:
          file_paths - Array of file paths to read from
          img_rows - The image height.
          img_cols - The image width.
          as_grey - Read the image as Greyscale or RGB.
          channels - Number of channels.
        @returns:
          The created and compiled model (Model)        
    """
    images = []
  
    for file_path in file_paths:
        images.append(imread(file_path, as_gray = False))
    
    images = np.asarray(images, dtype=np.float32)
  
  # normalize
    images = images / 255.0 #np.max(images)
  
  # reshape to match Keras expectaions
    images = images.reshape(images.shape[0], img_rows, img_cols, channels)

    return images

In [338]:
#### Hyper Parameters ####

## pixel if images ##
img_rows, img_cols = 120, 120
as_gray=True
in_channels = 3 # rgb files
num_classes = 0 # number of charts

batch_size = 32
epochs = 10
input_shape = (img_rows, img_cols, in_channels)
input_img = Input(shape= input_shape)
input_shape

(120, 120, 3)

In [339]:
## read the images from CSV File ####

x_train_amplitude=read_data(train_df.amplitude.values, img_rows, img_cols, in_channels)
x_train_phase = read_data(train_df.phase.values, img_rows, img_cols, in_channels)

# labels - convert class vectors to binary class matrices One Hot Encoding
labels = train_df.appliance.values/2
# labels = keras.utils.to_categorical(labels, num_classes)

In [340]:
# split the train data for test batches

x_train_comp = np.stack((x_train_amplitude, x_train_phase), axis=4)

x_train, x_test, y_train, y_test = train_test_split(x_train_comp, labels, test_size=0.2, random_state=666)

# take them apart
x_train_amplitude = x_train[:,:,:,:,0]
x_test_amplitude = x_test[:,:,:,:,0]

x_train_phase = x_train[:,:,:,:,1]
x_test_phase = x_test[:,:,:,:,1]

print(np.shape(x_test_phase))

(1, 120, 120, 3)


In [341]:
y_train.shape

(4,)

In [342]:
# X_train, X_test, y_train, y_test = train_test_split(x_train_amplitude, labels, test_size=0.2)

In [343]:
def create_convolution_layers(input_img):
    
    model = Conv2D(32, (3, 3), padding='same', input_shape=input_shape)(input_img)
    model = LeakyReLU(alpha=0.1)(model)
    model = MaxPooling2D((2, 2),padding='same')(model)
    model = Dropout(0.25)(model)
  
    model = Conv2D(64, (3, 3), padding='same')(model)
    model = LeakyReLU(alpha=0.1)(model)
    model = MaxPooling2D(pool_size=(2, 2),padding='same')(model)
    model = Dropout(0.25)(model)
    
    model = Conv2D(128, (3, 3), padding='same')(model)
    model = LeakyReLU(alpha=0.1)(model)
    model = MaxPooling2D(pool_size=(2, 2),padding='same')(model)
    model = Dropout(0.4)(model)


    return model

In [344]:
ampl_input = Input(shape=input_shape)
ampl_model = create_convolution_layers(ampl_input)

phase_input = Input(shape=input_shape)
phase_model = create_convolution_layers(phase_input)

conv = concatenate([ampl_model, phase_model])

conv = Flatten()(conv)

dense = Dense(512)(conv)
dense = LeakyReLU(alpha=0.1)(dense)
dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='softmax')(dense)

model = Model(inputs=[ampl_input, phase_input], outputs=output)

opt = optimizers.Adam()

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
# model.summary()
# np.shape(conv)

In [345]:
### Train the model###

best_weights_file="weights.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks = [checkpoint]


model.fit([x_train_amplitude, x_train_phase], y_train,
         batch_size=batch_size,
         epochs=epochs,
         callbacks=callbacks,
         verbose=1,
         validation_data=([x_test_amplitude, x_test_phase] ,y_test),
         shuffle=True
         )

AttributeError: 'Tensor' object has no attribute 'assign'

In [291]:
### convolutional base 
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

## feed conv base into dense layer 
## flatten the base to 1D
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1))

In [293]:
opt = optimizers.Adam()

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)

history=model.fit(x_train, y_train,
         batch_size=batch_size,
         epochs=100,
         verbose=1,
         validation_data=(X_test ,y_test),
         shuffle=True
         )

Train on 4 samples, validate on 1 samples
Epoch 1/100
4/4 [==============================] - 2s 377ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 65ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 66ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 64ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 68ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
4/4 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/100
4/4 [==============================] - 0s 65ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/100
4/4 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epo

Epoch 78/100
4/4 [==============================] - 0s 70ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 79/100
4/4 [==============================] - 0s 58ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 80/100
4/4 [==============================] - 0s 58ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 81/100
4/4 [==============================] - 0s 81ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 82/100
4/4 [==============================] - 0s 71ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 83/100
4/4 [==============================] - 0s 65ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 84/100
4/4 [==============================] - 0s 61ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 85/100
4/4 [==============================] - 0s 59ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 86/100
4/4 [==============================] - 0s 66ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 87/100
4/4 [===================